# EXPECTATION-MAXIMIZATION ALGORITHM FOR INCOMPLETE MATRIX WITH MATRIX COMPLETION,  RMSE COMPUTATION AND BIC CALCULATION

## 0. Import Packages & Set up Gaussian Formatting

In [4]:
import numpy as np
from scipy.stats import norm
from scipy.special import logsumexp
import sklearn.metrics
from matplotlib import pyplot as plt
from matplotlib.patches import Circle, Arc

In [194]:
class GaussianMixture(NamedTuple):
    """Tuple holding a gaussian mixture"""
    mu: np.ndarray  # (K, d) array - each row corresponds to a gaussian component mean
    var: np.ndarray  # (K, ) array - each row corresponds to the variance of a component
    p: np.ndarray  # (K, ) array = each row corresponds to the weight of a component

## 1. estep: Compute Posterior Probabilities and Log-Likelihood

In [258]:
def estep_em(X: np.ndarray, mixture: GaussianMixture) -> Tuple[np.ndarray, float]:
    """E-step: Softly assigns each datapoint to a gaussian component

    Args:
        X: (n, d) array holding the data, with incomplete entries (set to 0)
        mixture: the current gaussian mixture

    Returns:
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples
        float: log-likelihood of the assignment

    """
    N, D = X.shape
    K, _ = mixture.mu.shape
    mu, var, pi = mixture
    identifier = np.where(X>=1, 1, 0)
    post_log = np.zeros((N, K))

    for k in range(K):
        post_log[:,k] = np.sum(np.multiply(np.log(norm.pdf(X,mu[k],var[k]**0.5)),identifier),axis=1)

    # soft counts
    # p(point x(i) was generated by cluster j|x(i),θ)≜p(j∣i)=pjN(x(i);μ(j),σ2jI)p(x(i)∣θ).
    pi_log = np.log(pi)
    r_log = (pi_log+post_log)-logsumexp(pi_log+post_log, axis=1)[:,None]
    r = np.exp(r_log)
    
    # log MLE
    # ℓ^(x(1),…,x(n)∣θ)≜∑i=1n∑j=1Kp(j∣i)log(p(x(i) and x(i) generated by cluster j∣θ)p(j∣i)).    
    l = np.sum((pi_log+post_log-r_log)*r)
    
    return r,l
    raise NotImplementedError

## 2. mstep: Compute the Optimized Parameters of the GaussianMixture 

In [259]:
def mstep_em(X: np.ndarray, post: np.ndarray, mixture: GaussianMixture,
          min_variance: float = .25) -> GaussianMixture:
    """M-step: Updates the gaussian mixture by maximizing the log-likelihood
    of the weighted dataset

    Args:
        X: (n, d) array holding the data, with incomplete entries (set to 0)
        post: (n, K) array holding the soft counts
            for all components for all examples
        mixture: the current gaussian mixture
        min_variance: the minimum variance for each gaussian

    Returns:
        GaussianMixture: the new gaussian mixture
    """
    r = post
    N, D = X.shape
    K = r.shape[1]       
    identifier = np.where(X>=1, 1, 0)
    Cu_count = np.sum(identifier,axis=1)
    
   # mean (K,D) - -only update if denominator >=1
    mean = mixture[0]
    new_mean = np.where(np.dot(r.T,X)/(np.dot(r.T,identifier)+1e-16)<np.dot(r.T,X),np.dot(r.T,X)/(np.dot(r.T,identifier)+1e-16),mean)

    # var (K,) -- only update if min_var > 0.25  
    new_var = np.zeros((1,K))    
    for k in range(K):
        var_numerator = np.dot(r[:,k].T,np.sum(np.where(np.isnan(np.multiply(identifier,(X-new_mean[k,:])**2))==True,0,np.multiply(identifier,(X-new_mean[k,:])**2)),axis=1))
        var_denominator = np.dot(r[:,k].T,Cu_count)
        new_var[0,k] = var_numerator/var_denominator        
    new_var = np.array([max(min_variance,i) for i in new_var[0]])

    # weights (K,) -- (4,) array
    new_pi = r.sum(axis = 0)/N
    
    return GaussianMixture(new_mean,new_var,new_pi)
    raise NotImplementedError

## 3. Run the estep and mstep until the Absolute Optimization is Reached

In [260]:
def run_em(X: np.ndarray, mixture: GaussianMixture,
        post: np.ndarray) -> Tuple[GaussianMixture, np.ndarray, float]:
    """Runs the mixture model

    Args:
        X: (n, d) array holding the data
        post: (n, K) array holding the soft counts
            for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples
        float: log-likelihood of the current assignment
    """
    prev_l = None
    l = None
    while (prev_l is None or  l - prev_l > 1e-6*np.absolute(l)):
        prev_l = l
        r, l = estep_em(X, mixture)
        mixture = mstep_em(X, r, mixture, 0.25)

    return mixture, r, l
    raise NotImplementedError

## 4. Fill the Incomplete Matrix

In [243]:
def fill_matrix(X: np.ndarray, mixture: GaussianMixture) -> np.ndarray:
    """Fills an incomplete matrix according to a mixture model

    Args:
        X: (n, d) array of incomplete data (incomplete entries =0)
        mixture: a mixture of gaussians

    Returns
        np.ndarray: a (n, d) array with completed data
    """
    X1 = X.copy()  
    N, D = X.shape
    K, _ = mixture.mu.shape
    mu, var, pi = mixture
    identifier = np.where(X>=1, 1, 0)
    post_log = np.zeros((N, K))

    for k in range(K):
        post_log[:,k] = np.sum(np.multiply(np.log(norm.pdf(X,mu[k],var[k]**0.5)),identifier),axis=1)

    pi_log = np.log(pi)
    r_log = (pi_log+post_log)-logsumexp(pi_log+post_log, axis=1)[:,None]
    r = np.exp(r_log)
    
    for n in range(N):
        hu = [j[0]  for j in list(enumerate(X1[n,:])) if j[1] ==0]
        for i in hu:
            X1[n,i] = np.dot(r[n,:],mu[:,i])
            
    return X1        
    raise NotImplementedError

## 5. RMSE Calculation

In [268]:
np.sqrt(sklearn.metrics.mean_squared_error(X, X1))

NameError: name 'X' is not defined

## 6. BIC Calculatioin for K Selection

In [ ]:
def bic(X: np.ndarray, mixture: GaussianMixture,
        log_likelihood: float) -> float:
    """Computes the Bayesian Information Criterion for a
    mixture of gaussians

    Args:
        X: (n, d) array holding the data
        mixture: a mixture of spherical gaussian
        log_likelihood: the log-likelihood of the data

    Returns:
        float: the BIC for this mixture
    """
    N = X.shape[0]
    K, D = mixture.mu.shape 
    
    # free parameters = K(#pi) + K(#var) + K*D(#mu) - 1
    
    p = K+K+K*D-1    
    BIC = log_likelihood - p*np.log(N)/2
    
    return BIC
    raise NotImplementedError